# Introduction to the pandapower control module

This tutorial introduces the pandapower controle module with the example of tap changer control. For this, we first load the MV oberrhein network that contains two 110/20 kV transformers:

In [ ]:
# Importing necessary packages
from pandapower.run import runpp
from pandapower.networks import mv_oberrhein

net = mv_oberrhein()
net.trafo

If we run a power flow, we can see the voltage at the low voltage side of the transformers:

In [ ]:
runpp(net)
net.res_trafo.vm_lv_pu

Both transformers include a tap changer with a range of -9 to +9, which are set to positions -2 and -3 respectively:

In [ ]:
net.trafo["tap_pos"]

The tap position is constant within a power flow calculation. A controller can now be used to control the tap changer position depending on the bus voltage.

### Discrete Tap Control

The DiscreteTapControl from the pandapower control package receives a deadband of permissable voltage and uses the tap changer to keep the voltage within this voltage band. We define such a controller for the first transformer in the oberrhein network with a deadband of 0.99 to 1.01pu:

In [ ]:
import pandapower.control as control
trafo_controller = control.DiscreteTapControl(net=net, element_index=114, vm_lower_pu=0.99, vm_upper_pu=1.01)

The initiated controller automatically registers in the net. It can be found in the controller table:

In [ ]:
net.controller

We now run a controlled power flow by setting **run_control=True** within the runpp arguments and check the transformer voltage:

In [ ]:
# running a control-loop
runpp(net, run_control=True)
net.res_trafo.vm_lv_pu

The voltage at transformer 114 is now within the given range. If we checke the transformer table, we can see that the tap position of the first transformer as been changed from -2 to -1:

In [ ]:
net.trafo["tap_pos"]

In [ ]:
net.trafo.std_type

### Discrete Tap Control: Alternative mode

In [ ]:
net.controller.drop([trafo_controller.index], inplace = True)

The tap step percent is an alternative way to Discrete Tap Control. It sets a single voltage instead of a deadband of permissable voltage and calculates the voltage limit for the upper and lower side by adding and subtracting the vm_delta_pu respectively. We define such a controller for the first transformer as an alternative to Discrete Tap Control in the oberrhein network with a voltage of 1.05 pu:

In [ ]:
vm_delta_pu = net.trafo.at[142, "tap_step_percent"] / 100. * .5
vm_lower_pu = 1.05 - vm_delta_pu
vm_upper_pu = 1.05 + vm_delta_pu
print(vm_delta_pu,vm_lower_pu,vm_upper_pu)

We got the limit of upper side voltage by adding the vm_delta_pu to the set voltage and the lower side voltage limit by substracting the vm_delta_pu from the vm_set_pu.

In [ ]:
trafo_controller2 = control.DiscreteTapControl.from_tap_step_percent(
    net=net, element_index=114, vm_set_pu=1.05, side="lv", tol=1e-3, in_service=True, order=0,
    drop_same_existing_ctrl=False, matching_params=None)

Here we added the tap step percent to the first transformer

In [ ]:
runpp(net, run_control=True)

In [ ]:
net.controller

In [ ]:
net.res_trafo.vm_lv_pu

The voltage at transformer 114 is now changed to 1.048740 which is between the upper and lower limits

In [ ]:
net.res_trafo

In [ ]:
net.trafo["tap_pos"]

The tap position of the second transformer has been changed from -1 to -4:

### Continous Tap Control

It is also possible to control transformer with a **ContiniousTapControl** strategy. Instead of a range, this type of controller is able to achieve an exact output voltage. For this it assumes tap positions as floating numbers. We define such a controller for the second transformer in the network:

In [ ]:
trafo_controller = control.ContinuousTapControl(net=net, element_index=142, vm_set_pu=0.98, tol=1e-6)

If we now run the result, the low voltage side of the second transformer is controlled to exactly 0.98 pu:

In [ ]:
runpp(net, run_control=True)
net.res_trafo.vm_lv_pu

The tap position is set to -0.07:

In [ ]:
net.trafo["tap_pos"]

While this obviously would not possible in real transformers, it can be useful to assume continous taps in large scale studies to avoid big steps in the results.